### R Code to create data

In [1]:
%load_ext rpy2.ipython

ModuleNotFoundError: No module named 'rpy2'

In [ ]:
%%R -o powerData
library(pwr)
no.of.samples=seq(10,100,10)
pvals = c(.01,.05)
effect_size_2_temp = {}
effect_size_2 = {}
pvalues = c()
samples = c()
functions = c()
power = c()
powerExp = seq(from=0.1, to = .99, by = 0.01)
for (pv in pvals) {
    for (x in no.of.samples){
        for (pe in powerExp) {
            effect_size_2_temp = (pwr.2p.test(n=x, sig.level = pv, power = pe))$h
            functions = c(functions,effect_size_2_temp)
            pvalues = c(pvalues,pv)
            samples = c(samples,x)
            power = c(power,pe)
        }
    }
}
powerData <- data.frame(samples,functions,pvalues,power)

### Python code to format data

In [ ]:
import pandas as pd
import json
outputToHTML = []
document = powerData.to_csv().split("\n");
del document[len(document)-1]
header = document[0].split(",")[1:5]
del document[0]
tuples = {}
for row in document:
    row = row.split(",")[1:5]
    sampleNum = row[0]
    linePoint = row[1]
    pval = row[2]
    power = row[3]
    if (pval,sampleNum) not in tuples:
        tuples[(pval,sampleNum)] = {}
        tuples[(pval,sampleNum)]["pval"] = pval
        tuples[(pval,sampleNum)]["sample"] = sampleNum
        tuples[(pval,sampleNum)]["points"] = []
    tuples[(pval,sampleNum)]["points"].append([power,linePoint])


In [ ]:
j = "" # JSON object
j = j + "{"    
j = j + """\t"nodes": ["""
for t in tuples:
    j = j + str(json.dumps(tuples[t])) + ",\n"

j = j[:-2]
j = j + "\t]\n"
j = j + "}"

from IPython.display import HTML
from IPython.display import Javascript
Javascript("""window.data={};""".format(j))

### Javascript Code to create visualization

In [ ]:
%%javascript
require(["d3"], function(d3) {
  data.nodes.forEach(function(d,i){
      data.nodes[i].sample = +data.nodes[i].sample;
      data.nodes[i].pval = +data.nodes[i].pval;
      data.nodes[i].points.forEach(function(e,j){
          data.nodes[i].points[j][0] = +data.nodes[i].points[j][0];
          data.nodes[i].points[j][1] = +data.nodes[i].points[j][1];
      })
  })  
})

In [ ]:
%%javascript
require.config({paths: {d3: "d3.v5.min",}}); // Home directory of D3.JS: https://d3js.org/

require(["d3"], function(d3) {
    window.d3 = d3;
    
})



### Switching between p-value = .01

In [ ]:
from IPython.display import Javascript
Javascript('pwrChart1.js')

In [ ]:
%%javascript
window.redraw = function(slidervalue){
    require(["d3"], function(d3) {
        d3.select("div#sliderValue").text("Sample Size "+slidervalue)
        const svg = d3.select("div#graph");
        const updatePlot = data.nodes.filter(function(d,i) { return ((d.pval == .01)&&(d.sample == parseInt(slidervalue))) })
        svg.selectAll("g.node").data(updatePlot).selectAll("circle").data(function(d) { return d.points})
            .transition("2")
            .duration(1000)
            .delay(function(d,i) { return i * 25; })
            .attr("cx", d => x(d[1]))
            .attr("cy", d => y(d[0]))
        svg.selectAll("g.links").data(updatePlot)
            .selectAll("line.area").data(function(d) { return d.points })
            .transition("area").duration(1000)
            .delay(function(d,i) { return i * 25; })
            .attr("x1", function(d,i) { return d3.select(this).attr("x1"); })
            .attr("y1", function(d,i) { return d3.select(this).attr("y1"); })
            .attr("x2", function(d,i) { return x(d[1]); })
            .attr("y2", function(d,i) { return y(d[0]); })
    })
}

In [ ]:
HTML("""<center><input type="range" name="num" min="10" max="100", step="10" style="width:200px", onchange="redraw(this.value);" value="10"><div id="sliderValue">Sample Size 10</div></center>""")

### Switching between p-value = .05

In [ ]:
from IPython.display import Javascript
Javascript('pwrChart2.js')

In [ ]:
HTML("""<center><input type="range" name="num" min="10" max="100", step="10" style="width:200px", onchange="redraw2(this.value);" value="10"><div id="sliderValue2">Sample Size 10</div></center>""")

In [ ]:
%%javascript
window.redraw2 = function(slidervalue){
    require(["d3"], function(d3) {
        d3.select("div#sliderValue2").text("Sample Size "+slidervalue)
        const svg = d3.select("div#graph2");
        const updatePlot = data.nodes.filter(function(d,i) { return ((d.pval == .01)&&(d.sample == parseInt(slidervalue))) })
        svg.selectAll("g.node").data(updatePlot).selectAll("circle").data(function(d) { return d.points})
            .transition("2")
            .duration(1000)
            .delay(function(d,i) { return i * 25; })
            .attr("cx", d => x(d[1]))
            .attr("cy", d => y(d[0]))
        svg.selectAll("g.links").data(updatePlot)
            .selectAll("line.area").data(function(d) { return d.points })
            .transition("area").duration(1000)
            .delay(function(d,i) { return i * 25; })
            .attr("x1", function(d,i) { return d3.select(this).attr("x1"); })
            .attr("y1", function(d,i) { return d3.select(this).attr("y1"); })
            .attr("x2", function(d,i) { return x(d[1]); })
            .attr("y2", function(d,i) { return y(d[0]); })
    })
}